In [53]:
%reload_ext autoreload
%autoreload 2
import sys
import os
sys.path.append(os.path.dirname(os.getcwd()))

import os
import pandas as pd

import unicodedata
from datetime import datetime
from src.module.root.postgre import PostgreSQL
from src.module.root.datacontainer import DirectoryHandler
from src.module.dataextractor import Extractor
from src.module.dataformatter import Modifier

In [54]:
# 원천데이터 경로 파악
data_dir = DirectoryHandler.find_directory(
    start_path=os.getcwd(),
    target_directory_name='main'
)

In [55]:
import unicodedata

# 폴더 이름 변경
dir_eng = {
    '이벤트': 'event',
    '검색어': 'search',
    '주문결제': 'order',
    '회원정보': 'member'
}

directory = [dir for dir in os.listdir(data_dir) if '.DS_Store' not in dir ]

for dir in directory:
    original_dir = f'{data_dir}/{dir}'
    if os.path.isdir(original_dir):
        # 폴더 이름과 딕셔너리 키를 유니코드 정규화
        normalized_dir = unicodedata.normalize('NFC', dir)
        if normalized_dir in dir_eng:
            new_dir = f'{data_dir}/{dir_eng[normalized_dir]}'
            os.rename(original_dir, new_dir)
        else:
            print(f"'{dir}'에 해당하는 새로운 이름이 없습니다.")


'order'에 해당하는 새로운 이름이 없습니다.
'member'에 해당하는 새로운 이름이 없습니다.
'handled'에 해당하는 새로운 이름이 없습니다.
'search'에 해당하는 새로운 이름이 없습니다.
'event'에 해당하는 새로운 이름이 없습니다.


In [58]:
# 디렉토리별 데이터 경로 딕셔너리에 저장
file_dict = dict()
for root, dirs, files in os.walk(data_dir):
    key = os.path.basename(root)
    file_path = [f'{root}/{file}' for file in files]
    file_dict[key] = file_path

- 주문정보 추출

In [59]:
""" '주문정보' 데이터 추출 및 처리 """
order = file_dict['order']

# 데이터 추출
order_log = pd.DataFrame()
payment_log = pd.DataFrame()

for file_name in order:

    print(file_name + ' Start')
    
    # ~$ (임시 저장 테이블 제외)
    if os.path.basename(file_name).startswith('~$'):
        continue 

    if os.path.basename(file_name).startswith('.DS_Store'):
        continue 

    data = Extractor.data_load(path=file_name, header='조합원코드')
    sheets = list(data.keys())
    
    for sheet in sheets:
        if '주문' in unicodedata.normalize('NFC', file_name):
            order_log = pd.concat([order_log, data[sheet]], axis=0)
        elif '결제' in unicodedata.normalize('NFC', file_name):
            payment_log = pd.concat([payment_log, data[sheet]], axis=0)

# 컬럼명 변경
order_log_mapping = {
    '조합원코드': 'member_code',
    '구매날짜': 'purchase_date',
    '결제번호': 'payment_number',
    '상품코드': 'product_code',
    '상품명': 'product_name',
    '카테고리(대)': 'category_large',
    '카테고리(중)': 'category_medium',
    '단가': 'unit_price',
    '수량': 'quantity',
    '금액': 'amount'
}
order_log.rename(columns=order_log_mapping, inplace=True)

payment_log_mapping = {
    '조합원코드': 'member_code',
    '구매날짜': 'purchase_date',
    '결제번호': 'payment_number',
    '배송지 (시군구)': 'delivery_district',
    '금액': 'amount',
    '수매선수금 사용 금액': 'prepayment_used_amount',
    '마일리지 사용금액': 'mileage_used_amount',
    '쿠폰사용여부': 'coupon_used',
    '구매당시 회원 등급': 'member_grade_at_purchase',
    '접속유형': 'access_type'
}
payment_log.rename(columns=payment_log_mapping, inplace=True)


# 날짜 변환 함수 정의
def excel_date_to_datetime(value):
    try:
        # NaN 또는 유효하지 않은 값은 NaT로 처리
        if pd.isna(value):
            return pd.NaT
        # 유효한 숫자를 정수로 변환 후 날짜 처리
        return datetime.fromordinal(datetime(1900, 1, 1).toordinal() + int(value) - 2)
    except Exception as e:
        # 변환 실패 시 NaT 반환
        return pd.NaT

# purchase_date 컬럼 정리 및 변환
order_log['purchase_date'] = order_log['purchase_date'].apply(excel_date_to_datetime)
payment_log['purchase_date'] = payment_log['purchase_date'].apply(excel_date_to_datetime)

# 결과 저장
try:
    # 디렉토리 찾기
    handling_path = DirectoryHandler.find_directory(
        start_path=os.getcwd(),
        target_directory_name='handled'
    )

    # 디렉토리가 없으면 생성
    if handling_path is None:
        handling_path = '/Users/home/Desktop/file/online_mall/data/main/handled'
        os.makedirs(handling_path, exist_ok=True)

    # 데이터 저장
    order_log.to_csv(f'{handling_path}/order_log.csv', encoding='utf-8', index=False)
    payment_log.to_csv(f'{handling_path}/payment_log.csv', encoding='utf-8', index=False)

except Exception as e:
    print(f"Error saving files: {e}")

# PostgreSQL 업로드
try:
    work_dir = DirectoryHandler.find_directory(
        start_path=os.getcwd(),
        target_directory_name='online_mall'
    )
    config_path = f'{work_dir}/private_configs.yaml'

    postgre = PostgreSQL(config_file_path=config_path, request_url='local_postgre')
    postgre.connect()
    postgre.exec_sql("create schema if not exists online_mall;")
    postgre.write_data(data=order_log, table_name='order_log', schema='online_mall')
    postgre.write_data(data=payment_log, table_name='payment_log', schema='online_mall')
    postgre.close()
except Exception as e:
    print(f"Error uploading data to PostgreSQL: {e}")


/Users/home/Desktop/file/online_mall/data/main/order/연구소_1118_주문정보_7.xlsb Start
/Users/home/Desktop/file/online_mall/data/main/order/.DS_Store Start
/Users/home/Desktop/file/online_mall/data/main/order/연구소_1118_주문정보_6.xlsb Start
/Users/home/Desktop/file/online_mall/data/main/order/연구소_1118_주문정보_11.xlsb Start
/Users/home/Desktop/file/online_mall/data/main/order/연구소_1118_주문정보_1.xlsb Start
/Users/home/Desktop/file/online_mall/data/main/order/연구소_1118_주문정보_10.xlsb Start
/Users/home/Desktop/file/online_mall/data/main/order/연구소_1118_결제정보_1.xlsb Start
/Users/home/Desktop/file/online_mall/data/main/order/연구소_1118_주문정보_3.xlsb Start
/Users/home/Desktop/file/online_mall/data/main/order/연구소_1118_결제정보_2.xlsb Start
/Users/home/Desktop/file/online_mall/data/main/order/연구소_1118_주문정보_2.xlsb Start
/Users/home/Desktop/file/online_mall/data/main/order/연구소_1118_주문정보_5.xlsb Start
/Users/home/Desktop/file/online_mall/data/main/order/연구소_1118_주문정보_9.xlsb Start
/Users/home/Desktop/file/online_mall/data/main/or

- 검색어 데이터 추출

In [61]:
search = file_dict['search']

# 데이터 추출
order_keyword = pd.DataFrame()
search_keyword = pd.DataFrame()

for file_name in search:
    
    print(file_name + ' Start')

    # ~$ (임시 저장 테이블 제외)
    if os.path.basename(file_name).startswith('~$'):
        continue 

    if not file_name.endswith(('.xlsx')):    
        continue
    
    data = Extractor.data_load(path=file_name, header=None)
    sheets = list(data.keys())

    for sheet in sheets:
        if '주문' in unicodedata.normalize('NFC', sheet):
            order_keyword = pd.concat([order_keyword, data[sheet]], axis=0)
        elif '검색어' in unicodedata.normalize('NFC', sheet):
            search_keyword = pd.concat([search_keyword, data[sheet]], axis=0)
        
# 컬럼명 변경
order_keyword_mapping = ['member_code', 'order_date', 'ordered_keyword']
order_keyword.columns = order_keyword_mapping

search_keyword_mapping = ['member_code', 'search_date', 'searched_keyword']
search_keyword.columns = search_keyword_mapping

# csv 파일 반출
try:
    # 디렉토리 찾기
    handling_path = DirectoryHandler.find_directory(
        start_path=os.getcwd(),
        target_directory_name='handled'
    )

    # handling_path가 None이면 에러 발생
    if handling_path is None:
        raise FileNotFoundError("Target directory 'handled' not found.")
    
except FileNotFoundError:
    # 폴더가 없으면 생성
    handling_path = '/Users/home/Desktop/file/online_mall/data/main/handled'
    os.makedirs(handling_path, exist_ok=True)  # 폴더가 없을 경우 생성

order_keyword.to_csv(f'{handling_path}/order_keyword.csv', encoding='utf-8', index=False)
search_keyword.to_csv(f'{handling_path}/search_keyword.csv', encoding='utf-8', index=False)

# postgre 업로드
work_dir = DirectoryHandler.find_directory(
    start_path=os.getcwd(),
    target_directory_name='online_mall'
)
config_path = f'{work_dir}/private_configs.yaml'

postgre = PostgreSQL(config_file_path=config_path, request_url='local_postgre')
postgre.connect()
postgre.exec_sql("create schema if not exists online_mall;")
postgre.exec_sql("drop table if exists online_mall.order_keyword;")
postgre.exec_sql("drop table if exists online_mall.search_keyword;")
postgre.write_data(data=order_keyword, table_name='order_keyword', schema='online_mall')
postgre.write_data(data=search_keyword, table_name='search_keyword', schema='online_mall')
postgre.close()    

/Users/home/Desktop/file/online_mall/data/main/search/.DS_Store Start
/Users/home/Desktop/file/online_mall/data/main/search/온라인몰연구데이터(이용정보)_240724.xlsx Start
/Users/home/Desktop/file/online_mall/data/main/search/온라인몰연구데이터(이용정보_전체검색어)_240724_28.xlsx Start
/Users/home/Desktop/file/online_mall/data/main/search/온라인몰연구데이터(이용정보)_240923.xlsx Start
/Users/home/Desktop/file/online_mall/data/main/search/온라인몰연구데이터(이용정보)_240827.xlsx Start
/Users/home/Desktop/file/online_mall/data/main/search/온라인몰연구데이터(이용정보)_240930.xlsx Start
Initialize Class
Connected to PostgreSQL
PostgreSQL connection is closed


- 이벤트 (조합원 챌린지) 추출

In [62]:
event = file_dict['event']

# 데이터 추출
event_info = pd.DataFrame()

for file_name in event:

    # ~$ (임시 저장 테이블 제외)
    if os.path.basename(file_name).startswith('~$'):
        continue 
    
    if not file_name.endswith(('.xlsx', 'xlsb')):    
        continue
    
    data = Extractor.data_load(path=file_name, header='참여자코드')
    sheets = list(data.keys())

    for sheet in sheets:
        event_info = pd.concat([event_info, data[sheet]], axis=0)
  
# 컬럼명 변경
event_info_mapping = {
    '챌린지명 (샘플로 1개 챌린지 데이터 추출)': 'challenge_name',
    '참여자코드': 'participants'
}
event_info.rename(columns=event_info_mapping, inplace=True)

# csv 파일 반출
try:
    # 디렉토리 찾기
    handling_path = DirectoryHandler.find_directory(
        start_path=os.getcwd(),
        target_directory_name='handled'
    )

    # handling_path가 None이면 에러 발생
    if handling_path is None:
        raise FileNotFoundError("Target directory 'handled' not found.")
    
except FileNotFoundError:
    # 폴더가 없으면 생성
    handling_path = '/Users/home/Desktop/file/online_mall/data/main/handled'
    os.makedirs(handling_path, exist_ok=True)  # 폴더가 없을 경우 생성

event_info.to_csv(f'{handling_path}/event_info.csv', encoding='utf-8', index=False)

# postgre 업로드
work_dir = DirectoryHandler.find_directory(
    start_path=os.getcwd(),
    target_directory_name='online_mall'
)
config_path = f'{work_dir}/private_configs.yaml'

postgre = PostgreSQL(config_file_path=config_path, request_url='local_postgre')
postgre.connect()
postgre.exec_sql("create schema if not exists online_mall;")
postgre.write_data(data=event_info, table_name='event_info', schema='online_mall')
postgre.close()    

Initialize Class
Connected to PostgreSQL
PostgreSQL connection is closed


- 회원정보 추출

In [73]:
member_info.head(10)

,조합원코드,회원상태,연령,성별,주거지,가입일자(실버),가입일자(골드),가입일자(프리미엄),가입동기,가구인원수,가구유형,기대하는 상품,회원유형,마케팅수신여부(자연드림몰),마케팅수신여부(조합),마케팅수신여부(의료사협,출자금,수매선수금,중복가입여부
0,317352014,가능,26021,미확인,제주특별자치도 제주시,1,38190,1,미확인,0,미확인,기타,골드,Z,N,N,1520500,NaN,N
1,317453788,가능,22981,미확인,경상북도 포항시 북구,1,37567,1,미확인,0,미확인,기타,골드,Z,N,N,1853600,NaN,N
2,317454980,가능,26312,미확인,경상북도 포항시 북구,1,37567,1,미확인,0,미확인,기타,골드,Z,N,N,1789600,NaN,N
3,317461256,가능,26618,미확인,경상북도 포항시 남구,1,37594,1,미확인,0,미확인,기타,골드,Z,N,N,1277600,NaN,N
4,317463932,가능,25784,미확인,경상북도 포항시 북구,1,37603,1,미확인,0,미확인,기타,골드,Z,N,N,1239000,NaN,N
5,317472663,가능,25917,미확인,경상북도 포항시 남구,1,37694,1,미확인,0,미확인,기타,골드,Z,N,N,728500,NaN,N
6,317478935,가능,24344,미확인,경상북도 포항시 남구,1,37715,1,미확인,0,미확인,기타,골드,Z,N,N,1257500,NaN,N
7,317484964,가능,25278,미확인,경상북도 포항시 북구,1,37722,1,미확인,0,미확인,기타,골드,Z,N,N,1616300,NaN,N
8,317488929,가능,24682,미확인,경상북도 포항시 북구,1,37637,1,미확인,0,미확인,기타,골드,Z,N,N,840400,NaN,N
9,317489343,가능,24717,미확인,경기도 부천시 소사구,1,37727,1,미확인,0,선택안함,기타,골드,Y,N,N,496500,NaN,N


In [74]:
member = file_dict['member']

member_info = pd.read_excel('/Users/home/Desktop/file/online_mall/data/main/member/연구소_1118_회원정보.xlsb')
# None인 컬럼 삭제        
member_info = member_info.drop([col for col in member_info.columns if col is None], axis=1)

# 컬럼명 변경
member_info_mapping = {
    '조합원코드': 'member_code',
    '회원상태': 'member_status',
    '연령': 'birth_date',
    '성별': 'gender',
    '주거지': 'residence',
    '가입일자(실버)': 'silver_join_date',
    '가입일자(골드)': 'gold_join_date',
    '가입일자(프리미엄)': 'premium_join_date',
    '가입동기': 'join_motivation',
    '가구인원수': 'household_size',
    '가구유형': 'household_type',
    '기대하는 상품': 'expected_product',
    '회원유형': 'member_type',
    '마케팅수신여부(자연드림몰)': 'marketing_consent_online_mall',
    '마케팅수신여부(조합)': 'marketing_consent_coop',
    '마케팅수신여부(의료사협': 'marketing_consent_medical',
    '출자금': 'investment',
    '수매선수금': 'prepayment',
    '중복가입여부(의료사협과생협)': 'duplicate_membership_medical_and_coop'
}
member_info.rename(columns=member_info_mapping, inplace=True)

# 날짜 환산
member_info['birth_date'] = member_info['birth_date'].map(
    lambda value: Modifier.excel_date_formatting(value)
)
member_info['silver_join_date'] = member_info['silver_join_date'].map(
    lambda value: Modifier.excel_date_formatting(value)
)
member_info['gold_join_date'] = member_info['gold_join_date'].map(
    lambda value: Modifier.excel_date_formatting(value)
)
member_info['premium_join_date'] = member_info['premium_join_date'].map(
    lambda value: Modifier.excel_date_formatting(value)
)

# csv 파일 반출
try:
    # 디렉토리 찾기
    handling_path = DirectoryHandler.find_directory(
        start_path=os.getcwd(),
        target_directory_name='handled'
    )

    # handling_path가 None이면 에러 발생
    if handling_path is None:
        raise FileNotFoundError("Target directory 'handled' not found.")
    
except FileNotFoundError:
    # 폴더가 없으면 생성
    handling_path = '/Users/home/Desktop/file/online_mall/data/main/handled'
    os.makedirs(handling_path, exist_ok=True)  # 폴더가 없을 경우 생성

member_info.to_csv(f'{handling_path}/member_info.csv', encoding='utf-8', index=False)

# postgre 업로드
work_dir = DirectoryHandler.find_directory(
    start_path=os.getcwd(),
    target_directory_name='online_mall'
)
config_path = f'{work_dir}/private_configs.yaml'

postgre = PostgreSQL(config_file_path=config_path, request_url='local_postgre')
postgre.connect()
postgre.exec_sql("create schema if not exists online_mall;")
postgre.write_data(data=member_info, table_name='member_info', schema='online_mall')
postgre.close()    

Initialize Class
Connected to PostgreSQL
PostgreSQL connection is closed
